# Représentation des réels et précision numérique

Ce TP  a pour but de vous faire comprendre  les limites de la précision des calculs numériques, et pour cela voir comment les nombres réels sont représentés dans un ordinateur.


Je vous propose dans un premer temps une expérimentation faisant apparaitre quelques phénomènes surprenant, ensuite un retour sur ces phénomènes, où on essaiera de comprendre leur cause. On terminera par une dernière illustration des problèmes de précision numérique.

## I. Quelques expérimentations numériques sur la notion de précision

### a. Un test d'égalité

In [ ]:
0.1 + 0.1 == 0.2

Cela semble raisonnable ; donc continuons :

In [ ]:
0.1 + 0.1 + 0.1 == 0.3

Et là c'est plus surprenant !

### b. Quand obtient-on zéro ?

Considérons maintenant les deux fonctions suivantes :

In [ ]:
def premier_zero():
    n = 0
    while 1/2**n != 0:
        n += 1
    return n
    
def premier_zero_2(a):
    n = 0
    while a+ 1/2**n != a:
        n += 1
    return n

On pourrait s'attendre à ce que les boucles dans ces deux fonctions tournent indéfiniment. Ce n'est pas le cas :

In [ ]:
premier_zero()

In [ ]:
premier_zero_2(1)

On constate donc que `2**(-1074) == 0` répond `False` alors que `2**(-1075) == 0` répond `True`.

De même `1 + 2**(-52) == 1` répond `False` alors que `1 + 2**(-53) == 1` répond `True`

Et si on recalcule `premier_zero_2(a)` pour d'autres valeurs de `a` ?

In [ ]:
for k in range(10):
    print(f"{k} -> {premier_zero_2(2**k)}") 

In [ ]:
for k in range(10):
    print(2**45 +1/2**k -2**45)

### c. Des grands nombres

In [ ]:
2 ** 1050

Mais avec des nombres en virgule flottante, ce n'est pas la même chose :

In [ ]:
2.0 ** 1023

In [ ]:
2.0 ** 1024

Il apparait donc qu'on peut regarder des entiers de taille indéfinie, mais que les nombres en virgule flottante ont une limite supérieure qui est de l'ordre de $10^{308}$.

### d. Des petits nombres

In [ ]:
2 ** -1074

In [ ]:
2 ** -1074 == 0

In [ ]:
2 ** -1075

In [ ]:
2 ** -1075 == 0

Un nombre en virgule flottante de l'ordre de $10^{-325}$ est donc considéré par python comme exactement nul.

### e. Un  calcul qui donne un résultat surprenant

Si on calcule l'expression mathématique suivante :

$$ 10^k \left( 3 ((10^k +\frac{1}{3}) -10^k)- 1\right)  $$

on trouve immédiatement 0.
Regardons ce que ça donne avec python, pour différentes valeurs de k.
On respecte ici le parenthésage donné dans la formule, sans faire de simplification.


In [ ]:
for k in range(15):
    print(k," -> ",(3 * ((10**k + 1/3)- 10**k) - 1) * 10**k )

## II. Autopsie des phénomènes observés

Il reste maintenant à comprendre tout ça, et pour cela, il est nécessaire de regarder en détail la représentation des nombres dans un ordinateur. C'est le moment de regarder la vidéo sur la représentation des nombres réels sur machine.


Pour investiguer les causes des phénomènes numériques que nous avons observés, je vous fournis une fonction `representation(x)` qui affiche de manière humainement compréhenseible la représentation machine d'un réel `x`. Cette représentation est consituée de trois informations :
- le signe (codé sur 1 bit)
- l'exposant de 2 (il est compris entre -1023 et +1023 et est codé sur 11 bits)
- la mantisse (c'est à dire les chiffres après la virgule flottante ; elle est codée sur 52 bits)

Je ne vous demande pas de comprendre la façon dont cette fonction est codée ; c'est juste un outil à utiliser.

In [ ]:
import struct

def representation(x):
    c = ''.join(f'{b:08b}' for b in struct.pack('>d', x))
    if c[0] == "0":
        signe = "+"
    else:
        signe = "-"
    print("signe    -> ",signe)
    print("exposant -> ",int(c[1:12],2) - 1023)
    print("mantisse -> ",c[12:])

Cette fonction s'utilise de la façon suivante :

In [ ]:
x = 12456
representation(x)

Cela signifie que $x$ s'écrit en notation binaire `1.1000010101  2**(13)`

Autrement dit : $x = 2^{13} + 2^{12} + 2^7 + 2^5 + 2^3$

**Recherche :**
Reprenez les exemples vus précédemment, et regardez à chaque étape du calcul la représentation machine des nombres qui entrent en jeu.
Essayez de comprendre précisément les causes des phénomènes observés.

## III. Un exercice sur les solutions d'une équation du second degré

On regarde une équation du second degré $ax^2+bx+c=0$. On sait qu'elle possèdes deux solutions réelles :

$$
x_1 = \frac{-b + \sqrt{b^2-4ac}}{2a}  \qquad  x_2 = \frac{-b - \sqrt{b^2-4ac}}{2a} 
$$

On note $f(x) = ax^2 +bx+c$.

**Question préliminaire :** Ecrire des fonctions `f(a,b,c,x)`, `x1(a,b,c)`, `x2(a,b,c)`


**Expérimentation numérique :** On fixe maintenant  `a = 1`, `b = -12345678`, `c = 2`.

Calculez les valeurs `x1` et `x2` de $x_1$ et $x_2$ données par ces formules, ainsi que leurs images par `f`.

Que constatez-vous ?

Calculez une nouvelle approximation `y2` de $x_2$ en utilisant le fait que $x_1x_2 = c$. Calculez $f(x_2)$ en utilisant cette nouvelle approximation.

Que constatez-vous ?
 
Que vaut aussi la différence `x2 -y2` ?

Essayez d'expliquer ce que vous avez constaté.
